In [1]:
import probtorch
import torch

import combinators
import hmm
import smc
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
mu = torch.zeros(1).unsqueeze(0)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs, mu, sigma, delta, generative = hmm.hmm_step(zs, mu, sigma, delta, t+1, generative)

In [5]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [6]:
num_particles = 500

In [7]:
smc_step = smc.smc(hmm.hmm_step, hmm.hmm_retrace)

In [8]:
hmm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(1.),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(1.),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(1.),
    }
}

In [9]:
inference, init_hmm = smc.variational_smc(num_particles, hmm.init_hmm, smc_step, 500, 50, hmm_params, generative, 5)

In [10]:
inference

Trace{'mu': Normal([torch.FloatTensor of size 500]), 'sigma': Normal([torch.FloatTensor of size 500]), 'delta': Normal([torch.FloatTensor of size 500]), 'Z_0': Normal([torch.FloatTensor of size 500]), 'Z_1': Normal([torch.FloatTensor of size 500]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 500]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 500]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 500]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 500]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 500]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 500]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 500]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 500]),

In [11]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - generative[key].value)**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 1.723170
SMC MSE at time 2: 0.842310
SMC MSE at time 3: 1.588546
SMC MSE at time 4: 0.186023
SMC MSE at time 5: 0.627453
SMC MSE at time 6: 0.478724
SMC MSE at time 7: 1.426999
SMC MSE at time 8: 0.316259
SMC MSE at time 9: 0.825677
SMC MSE at time 10: 0.915360
SMC MSE at time 11: 2.174628
SMC MSE at time 12: 1.263073
SMC MSE at time 13: 0.060577
SMC MSE at time 14: 0.422524
SMC MSE at time 15: 0.302408
SMC MSE at time 16: 5.797582
SMC MSE at time 17: 0.051837
SMC MSE at time 18: 0.252933
SMC MSE at time 19: 0.286296
SMC MSE at time 20: 6.142094
SMC MSE at time 21: 1.435743
SMC MSE at time 22: 0.713476
SMC MSE at time 23: 5.013797
SMC MSE at time 24: 0.762840
SMC MSE at time 25: 0.031600
SMC MSE at time 26: 1.622066
SMC MSE at time 27: 14.095006
SMC MSE at time 28: 0.146375
SMC MSE at time 29: 3.207798
SMC MSE at time 30: 0.003540
SMC MSE at time 31: 4.088219
SMC MSE at time 32: 0.251796
SMC MSE at time 33: 0.011031
SMC MSE at time 34: 1.880746
SMC MSE at time 35: 0.